In [49]:
# Pull rawdata
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler

data=pd.read_csv('Full_Data.csv')

C:\Users\rwang\DataBootCamp\AnaConda\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [50]:
data["HR/PA"] = data["HR_bat"]/data["PA"]
data["R/PA"] = data["R_bat"]/data["PA"]
data["RBI/PA"] = data["RBI"]/data["PA"]

In [51]:
data['MVP_Classification']=data['awardID']
data['MVP_Classification'] = data['MVP_Classification'].map({'MVP': 1})
data['MVP_Classification'] = data['MVP_Classification'].replace(np.nan, 0)

In [52]:
data_encoded = pd.get_dummies(data, columns=["Pos"])
data_encoded.head()

,Season,Name_bat,Team_bat,Age_bat,G_bat,PA,H_bat,1B,2B,3B,...,MVP_Classification,Pos_1B,Pos_2B,Pos_3B,Pos_C,Pos_CF,Pos_LF,Pos_P,Pos_RF,Pos_SS
0,1963,Buster Narum,Orioles,22.0,7.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,1,0,0
1,2010,Gustavo Chacin,Astros,29.0,44.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,1,0,0
2,1938,Bill Lefebvre,Red Sox,22.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,1,0,0
3,2008,Felix Hernandez,Mariners,22.0,31.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,1,0,0
4,2005,Eddie Rogers,Orioles,26.0,8.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0,0,0,0,0,0,0,0,1


In [53]:
years = data_encoded[['Season']].groupby('Season').sum().reset_index()
years_bucket_1=years[(years['Season']>=1969) & (years['Season']<=1983)]
years_bucket_2=years[(years['Season']>=1984) & (years['Season']<=2007)]
years_bucket_3=years[years['Season']>=2008]

In [66]:
# Create models for years_bucket_2
for i in years_bucket_2['Season']:
    # -----------------------------------------------------------------------------
    # Create Random Forest
    # Create datasets
    data_name = data_encoded[data_encoded['Season']==i]
    data_name=data_name[data_name['lgID_batting']=="NL"]
    data_name = data_name.fillna(value=0)
    y_name=data_name['MVP_Classification']
    X_name_encoded=data_name[['PA','HR/PA', 'R/PA', 'RBI/PA','SB',
                 'AVG','OBP', 'SLG','wOBA', 'WAR_bat','W_pit', 'L_pit', 'ERA', 'G_pit',
                 'CG', 'ShO', 'SV','IP', 
                 'K/9', 'BB/9', 'WHIP','FIP', 'WAR_pit','FP','Playoffs', 'Pos_1B', 'Pos_2B', 'Pos_3B',
                 'Pos_C', 'Pos_CF', 'Pos_LF', 'Pos_P', 'Pos_RF', 'Pos_SS']]
    X_scaler = StandardScaler().fit(X_name_encoded)
    X_name_encoded = pd.DataFrame(X_scaler.transform(X_name_encoded), columns=X_name_encoded.columns)

    # Create Random Forest
    rf = RandomForestClassifier(n_estimators=200)
    rf = rf.fit(X_name_encoded, y_name)
    rf.score(X_name_encoded, y_name)
    
    # Get feature importance
    importances = rf.feature_importances_
    list=sorted(zip(rf.feature_importances_, X_name.columns), reverse=True)
    # Top 10 feature importances
    list_10=list[:10]
    
    filename = str(i) + '_rf.sav'
    pickle.dump(rf, open(filename, 'wb'))
    
    # ------------------------------------------------------------------------------
    # Create Logistic Regression
    # Create datasets
    data_name_logit=data_name[data_name['pointsWon']>0]
    y_name_logit=data_name_logit['pointsWon']
    X_name_logit=data_name_logit[['PA','HR/PA', 'R/PA', 'RBI/PA','SB',
                 'AVG','OBP', 'SLG','wOBA', 'WAR_bat','W_pit', 'L_pit', 'ERA', 'G_pit',
                 'CG', 'ShO', 'SV','IP', 
                 'K/9', 'BB/9', 'WHIP','FIP', 'WAR_pit','FP','Playoffs', 'Pos_1B', 'Pos_2B', 'Pos_3B',
                 'Pos_C', 'Pos_CF', 'Pos_LF', 'Pos_P', 'Pos_RF', 'Pos_SS']]

    X_scaler = StandardScaler().fit(X_name_logit)
    X_name_logit = pd.DataFrame(X_scaler.transform(X_name_logit), columns=X_name_logit.columns)
    # Reshape y variable
    y_name_logit=y_name_logit.values.reshape(len(y_name_logit),1)
        
    # Create linear regression model
    logregr = linear_model.LogisticRegression()
    logregr.fit(X_name_logit,y_name_logit)
    
    # Save the model to disk
    filename = str(i) + '_linear.sav'
    pickle.dump(regr, open(filename, 'wb'))

C:\Users\rwang\DataBootCamp\AnaConda\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\rwang\DataBootCamp\AnaConda\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Create models for years_bucket_1
for i in range(len(years_bucket_1)):

In [ ]:
# Create models for years_bucket_3
for i in range(len(years_bucket_3)):